In [1]:
%%time

import pandas as pd
from datetime import datetime
import calendar

xls = pd.read_excel("./A股全部市值.xlsx")

CPU times: user 31.9 s, sys: 782 ms, total: 32.7 s
Wall time: 34.1 s


In [ ]:
xls.loc[xls["行业.1"]=="金融"].nlargest(10,datetime(2019,12,31))
# xls.loc[xls["行业.1"]=="金融"].sum()

In [23]:
from datetime import datetime
from functools import lru_cache
import numpy as np

@lru_cache(maxsize=None)
def get_topn_of_year( n, year, month, industry):
    df = xls
    column_index = datetime(year, month, calendar.monthrange(year, month)[1])
#     return df[df["行业.1"] == industry][["行业", "行业.1", column_index]].nlargest(n, column_index)
    return df[df["行业.1"] == industry][["行业", "行业.1", column_index]]

@lru_cache(maxsize=None)
def get_values_of_year( n, year, month):
    df=xls
    labels = ['金融', '农业食品', '能源资源', '医疗健康', '基础设施', '消费服务', '信息技术', '制造业',
       '房地产建筑', '化工材料', '物流', '文化教育']
    values = list(map(lambda l: int(get_topn_of_year( n, year, month, l).iloc[:, 2].sum()), labels))
    return values

@lru_cache(maxsize=None)
def get_values_to_year( n, index):
    df = xls
    values = np.array(get_values_of_year(n,1990,1))
    for i in range(1,index):
        
        date = datetime(1990,1,1)
        date = date + relativedelta(months=i)
        values += np.array(get_values_of_year(n, date.year, date.month))
    return values
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom',
                   size=15)

def autolabelh(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        width = rect.get_width()
        ax.annotate(f'{width:,}',
                    xy=(width, rect.get_y() + rect.get_height()/2),
                    xytext=(3, 0),  # 3 points horizontal offset
                    textcoords="offset points",
                    ha='left', va='center',
                   size=15)

In [191]:

labels = ['金融', '农业食品', '能源资源', '医疗健康', '基础设施', '消费服务', '信息技术', '制造业',
       '房地产建筑', '化工材料', '物流', '文化教育']
colors = []
# for l in labels:
#     print(l)
#     print(get_topn_of_year(xls, 20, 2019, 12, l).iloc[:, 2].sum())
values = list(map(lambda l: int(get_topn_of_year(xls, 22, 2019, 12, l).iloc[:, 2].sum()), labels))
# print(values)
get_values_of_year(xls, 20,1990,1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
%%time

import matplotlib.pyplot as plt
from matplotlib import font_manager
import matplotlib.animation as animation
from dateutil.relativedelta import relativedelta

fname = ['Songti SC'][0]
plt.rcParams['font.family']=[fname]
plt.rcParams['axes.unicode_minus']=False

fontP = font_manager.FontProperties()
fontP.set_family(fname)
fontP.set_size(14)

fontTitle = font_manager.FontProperties()
fontTitle.set_family(fname)
fontTitle.set_size(18)

fig = plt.figure(figsize=(6, 10))
ax  = fig.add_subplot(111,frameon=False)
ax.spines["top"].set_visible(False)    #隐藏边框
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
plt.gcf().subplots_adjust(left=0.15)


def draw(index):
    plt.cla()
#     plt.box(on=None)    # 去除边框
    plt.margins(x=0.2)  # 横着右边留白20%
    date = datetime(1990,1,1)
    date = date + relativedelta(months=index)
    ax.set_xlabel(u'估值(亿元人民币)', fontproperties=fontP)
    ax.set_title(u'{}年各行业数据'.format(date.year), fontproperties=fontTitle)
    
    # 第3列为投资额
    labels = ['金融', '农业食品', '能源资源', '医疗健康', '基础设施', '消费服务', '信息技术', '制造业', '房地产建筑', '化工材料', '物流', '文化教育']
#     colors = ['red', 'blue', 'pink', 'yellow', 'brown', 'purple', 'orange', 'cyan', 'grenn', 'orange', 'cyan', 'grenn']
    colors = ['teal','red','blue','gold','green','cyan','pink','purple','orange', 'magenta', 'violet', 'lime', 'olive', 'salmon','goldenrod','fuchsia', 'orchid', 'plum']
    
#     print(date)
    values = get_values_of_year( 20, date.year, date.month)
#     print(values)
    rects = plt.barh(labels,values, height=0.4, align='center', xerr=0.0001, color=colors)
    autolabelh(rects)
    plt.xlim(left=0)
    if not any(values):
        plt.xlim(right=1)
#     ax.set_xticklabels(labels)
    return rects

ani = animation.FuncAnimation(fig, draw, range(360), interval=300)

# ani.save('history.gif', fps=8, writer='imagemagick')
ani.save('history.mp4', fps=8, writer='imagemagick')
# FFwriter = animation.FFMpegWriter(fps=8, codec="libx264")     
# ani.save('basic_animation1.mp4', writer = FFwriter )

len(xls["行业.1"].unique())

In [58]:
import datetime
d = datetime.datetime(2019,12,31)
xls[["上市年","上市后市值涨幅", d]].describe()



,上市年,上市后市值涨幅,2019-12-31 00:00:00
count,3760.000000,3760.000000,3760.000000
mean,2008.125532,5.150307,171.025561
std,8.177317,26.745439,744.716129
min,1990.000000,0.000000,2.648646
25%,2000.000000,1.022804,29.975945
50%,2010.000000,1.755812,50.436484
75%,2016.000000,3.727025,109.147355
max,2019.000000,1261.998825,20518.100251


In [91]:
a = get_topn_of_year(xls,11,2019,12, "金融")
a
a.iloc[:,2].sum()

108774.90028737039

In [41]:
d = datetime.datetime(2019,12,31)
d

datetime.datetime(2019, 12, 31, 0, 0)

In [42]:
str(d)

'2019-12-31 00:00:00'

In [43]:
d - datetime.datetime(0,0,1,)

ValueError: year 0 is out of range

SyntaxError: invalid syntax (<ipython-input-44-7e21033584ae>, line 1)

In [163]:
import calendar
from datetime import timedelta
dt2 = datetime(2020,2, calendar.monthrange(2020,2)[1])
dt3 = dt2 + timedelta(months=12)
calendar.monthrange(2020,2)[1]

TypeError: 'months' is an invalid keyword argument for __new__()

In [164]:
import dateutil

In [165]:
from dateutil.relativedelta import relativedelta

In [166]:
dt1 = datetime(2020,2,29)

In [169]:
dt2 = dt1 + relativedelta(months=12)
dt3 = dt2 + relativedelta(months=12)

In [172]:
dt2,dt3,dt3.year,dt3.month

(datetime.datetime(2021, 2, 28, 0, 0),
 datetime.datetime(2022, 2, 28, 0, 0),
 2022,
 2)